In [16]:
import os
import sys
import signal
import requests
import selenium
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

In [17]:
class WebScraper:
    def __init__(self, folder_name = 'data'):
        self.CHROME_DRIVER_PATH = ".\chromedriver_win32\chromedriver.exe"
        # self.GECKO_DRIVER_PATH  = os.path.join('..','geckodriver')
                                               
        self.NEWS_IMG_PATH = os.path.join(folder_name, 'img')
        self.NEWS_PATH = folder_name

# TO DO:

### STEP 1
- Contents CSV 1 (Image Name, Caption, Inset Boolean)
- Contents CSV 2 (Article URL, Article Name, Categories, Date, List of Image Names)
- Draw ERD

### STEP 2
- Get most recent articles (example: past week or past 100 articles)
- Perform same functionality as in step 1
- Calculate statistics (To be discussed in further detail)

### STEP 3
- Research Image-in-Image and Picture-in-Picture Decomposition

In [18]:
m = WebScraper(folder_name = '.\\times_of_malta')

# def signal_handler(sig, frame):
#     pd.DataFrame(columns=['Title','Image Name','Caption','Body'], data=data).to_csv(os.path.join(m.NEWS_PATH,'data.csv'), index=False)
#     sys.exit()

def get_img_ext(img_link):
    if img_link.endswith('.jpeg'):
        return '.jpeg'
    elif img_link.endswith('.jpg'):
        return '.jpg'
    elif img_link.endswith('.png'):
        return '.png'
    else: return ""

options = webdriver.ChromeOptions()
options.add_experimental_option('excludeSwitches', ['enable-logging'])

service = Service(executable_path=m.CHROME_DRIVER_PATH)

# Since chromedriver is in PATH we dont have specify it location otherwise webdriver.Chrome('path/chromedriver.exe')
driver = webdriver.Chrome(service = service, options = options)

# Opening required website to scrape content 
driver.get('https://timesofmalta.com/search?keywords=inset&author=0&tags=0&sort=date&order=desc&fields%5B0%5D=title&fields%5B1%5D=body&page=1')


#Closing pop-ups
# print('Closing initial pop-ups: ',end='')
# driver.find_element(By.XPATH,'/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/button[1]').click()
# print('[OK]')


data = []
count = 0
article_index = 0
page_index = 0

#Setup ^C Handler to save on signal.
try: 
    # print(driver.find_element(By.CLASS_NAME,'fc-button fc-cta-consent fc-primary-button'))
    # print("FOUND BUT NOT CLICKED")
    WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, 
                                                                "//div[@class='fc-consent-root']//button[@aria-label='Consent']//p[@class='fc-button-label' and text()='Consent']"))).click()
    #driver.find_element(By.CLASS_NAME,"fc-button fc-cta-consent fc-primary-button").click()
except: 
    pass
for i in range(10):

    # signal.signal(signal.SIGINT, signal_handler)
    try: 
        #Remove donation message
        try: driver.find_element(By.XPATH,'//*[@id="eng-accept"]').click()
        except: pass

        #Click on article
        driver.find_element(By.XPATH,f'//*[@id="listing-articles"]/div[{str(2+article_index)}]/a').click()
        #Increment article index
        article_index += 1

        sleep(1)

        url = driver.current_url
        #Get Title
        title = driver.find_element(By.XPATH,'//*[@id="article-head"]/div/h1').text

        print(f'Title: {title}')
        
        #Get Thumbnail + Images
        img_links = [img for img in \
                     driver.find_elements(By.XPATH,'//*[@id="observer"]/main/article/div[2]/div/*/img') +\
                     driver.find_elements(By.XPATH,'//*[@id="article-head"]/div/picture/img')]
        
        captions = ""
        images   = ""

        #Write images to disk
        for img_link in img_links:
            img_src   = img_link.get_attribute('src')        #Get source
            captions  = img_link.get_attribute('alt') + '☺' #Append current caption

            img_ext  = get_img_ext(img_src)                #Get image extension
            img_name = f'img{str(count).zfill(5)}'+img_ext #Get image name
            images  = img_name + ','                      #Append image name to list
            img_data = requests.get(img_src).content       #Download image

            insetBool = "NO"
            if "inset" in captions or "Inset" in captions:
                insetBool = "YES"
            data.append([url, images[:-1], captions[:-1], insetBool])
            print('Saving...\n')
            pd.DataFrame(columns=['URL', 'Image Name', 'Caption', 'Inset'], data=data).to_csv(os.path.join(m.NEWS_PATH,'Article_Information.csv'), index=False)
            count += 1

            
            #Write current image to disk
            with open(os.path.join(m.NEWS_IMG_PATH,img_name),'wb') as f:
                f.write(img_data)

        #Get Body
        text_content = driver.find_element(By.XPATH,'//*[@id="observer"]/main/article/div[2]/div')                                                     
        body = " ".join(p.text for p in text_content.find_elements(By.TAG_NAME,'p'))

        #Add row
        #pd.DataFrame(columns=['URL', 'Article Name', 'Categories', 'Date', 'Images'], data=data).to_csv(os.path.join(m.NEWS_PATH,'Image_Information.csv'), index=False)

        #Save to csv
        # if i%50 == 0:
        

    #Go back
        print('Back to home page\n')
        driver.back()

    except selenium.common.exceptions.NoSuchElementException as e:
        #Switch to next page
        sleep(1.5)

        try: driver.find_element(By.XPATH,f'//*[@id="observer"]/main/div/div[2]/div/span[{str(2+page_index)}]').click()
        except: 
            print(f'Error: Reloading page {page_index+2}')
            driver.get(f'https://google.com')
            sleep(2)
            page_index += 1
            driver.get(f'https://timesofmalta.com/search?keywords=inset&author=0&tags=0&sort=date&order=desc&fields%5B0%5D=title&fields%5B1%5D=body&page={str(page_index+2)}') #Skip page

            try: driver.find_element(By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click() #Close pop-up
            except: pass

            article_index = 0 #Invoke next-page switch
            
            continue #Go back to beggining

        

        page_index   += 1
        article_index = 0
        print(f'Next Page! -> {page_index+1}')
    
    except selenium.common.exceptions.ElementClickInterceptedException as e:
        print('Click Intercepted - Skipping')
        article_index += 1
        continue

    except Exception as e:
        print(str(e))
        article_index += 1
        page_index += 1
        continue

Title: History - The canal/tunnel proposals to connect Grand Harbour and Marsamxett
Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

Saving...

5 columns passed, passed data had 4 columns
Error: Reloading page 3
Title: The week at a glance
Saving...

Saving...

Saving...

Saving...

Saving...

5 columns passed, passed data had 4 columns
Error: Reloading page 5
Title: Sharing local memories of two canonised pontiffs
Saving...

Saving...

Saving...

Saving...

Saving...

5 columns passed, passed data had 4 columns
Error: Reloading page 7
Title: Improving literacy through play
Saving...

5 columns passed, passed data had 4 columns
Error: Reloading page 9
Error: Reloading page 10
Error: Reloading page 11


In [19]:
# m = WebScraper(folder_name = '.\\times_of_malta')

# # def signal_handler(sig, frame):
# #     pd.DataFrame(columns=['Title','Image Name','Caption','Body'], data=data).to_csv(os.path.join(m.NEWS_PATH,'data.csv'), index=False)
# #     sys.exit()

# def get_img_ext(img_link):
#     if img_link.endswith('.jpeg'):
#         return '.jpeg'
#     elif img_link.endswith('.jpg'):
#         return '.jpg'
#     elif img_link.endswith('.png'):
#         return '.png'
#     else: return ""

# options = webdriver.ChromeOptions()
# options.add_experimental_option('excludeSwitches', ['enable-logging'])

# service = Service(executable_path=m.CHROME_DRIVER_PATH)

# # Since chromedriver is in PATH we dont have specify it location otherwise webdriver.Chrome('path/chromedriver.exe')
# driver = webdriver.Chrome(service = service, options = options)

# # Opening required website to scrape content 
# driver.get('https://timesofmalta.com/search?keywords=inset&author=0&tags=0&sort=date&order=desc&fields%5B0%5D=title&fields%5B1%5D=body&page=1')


# #Closing pop-ups
# # print('Closing initial pop-ups: ',end='')
# # driver.find_element(By.XPATH,'/html/body/div[4]/div[2]/div[1]/div[2]/div[2]/button[1]').click()
# # print('[OK]')


# data = []
# count = 0
# article_index = 0
# page_index = 0

# #Setup ^C Handler to save on signal.
# try: 
#     # print(driver.find_element(By.CLASS_NAME,'fc-button fc-cta-consent fc-primary-button'))
#     # print("FOUND BUT NOT CLICKED")
#     WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, 
#                                                                 "//div[@class='fc-consent-root']//button[@aria-label='Consent']//p[@class='fc-button-label' and text()='Consent']"))).click()
#     #driver.find_element(By.CLASS_NAME,"fc-button fc-cta-consent fc-primary-button").click()
# except: 
#     pass
# for i in range(10):

#     # signal.signal(signal.SIGINT, signal_handler)
#     try: 
#         #Remove donation message
#         try: driver.find_element(By.XPATH,'//*[@id="eng-accept"]').click()
#         except: pass

#         #Click on article
#         driver.find_element(By.XPATH,f'//*[@id="listing-articles"]/div[{str(2+article_index)}]/a').click()
#         #Increment article index
#         article_index += 1

#         sleep(1)

#         url = driver.current_url
#         #Get Title
#         title = driver.find_element(By.XPATH,'//*[@id="article-head"]/div/h1').text

#         print(f'Title: {title}')
        
#         #Get Thumbnail + Images
#         img_links = [img for img in \
#                      driver.find_elements(By.XPATH,'//*[@id="observer"]/main/article/div[2]/div/*/img') +\
#                      driver.find_elements(By.XPATH,'//*[@id="article-head"]/div/picture/img')]
        
#         captions = ""
#         images   = ""

#         #Write images to disk
#         for img_link in img_links:
#             img_src   = img_link.get_attribute('src')        #Get source
#             captions  += img_link.get_attribute('alt') + '☺' #Append current caption

#             img_ext  = get_img_ext(img_src)                #Get image extension
#             img_name = f'img{str(count).zfill(5)}'+img_ext #Get image name
#             images  += img_name + ','                      #Append image name to list
#             img_data = requests.get(img_src).content       #Download image

#             count += 1

#             #Write current image to disk
#             with open(os.path.join(m.NEWS_IMG_PATH,img_name),'wb') as f:
#                 f.write(img_data)

#         #Get Body
#         text_content = driver.find_element(By.XPATH,'//*[@id="observer"]/main/article/div[2]/div')                                                     
#         body = " ".join(p.text for p in text_content.find_elements(By.TAG_NAME,'p'))

#         #Add row
#         data.append([url, images[:-1], captions[:-1]])

#         #Save to csv
#         # if i%50 == 0:
#         print('Saving...\n')
#         for image in images:
#             pd.DataFrame(columns=['Image Name','Caption', 'Inset'], data=data).to_csv(os.path.join(m.NEWS_PATH,'contents.csv'), index=False)

#     #Go back
#         print('Back to home page\n')
#         driver.back()

#     except selenium.common.exceptions.NoSuchElementException as e:
#         #Switch to next page
#         sleep(1.5)

#         try: driver.find_element(By.XPATH,f'//*[@id="observer"]/main/div/div[2]/div/span[{str(2+page_index)}]').click()
#         except: 
#             print(f'Error: Reloading page {page_index+2}')
#             driver.get(f'https://google.com')
#             sleep(2)
#             page_index += 1
#             driver.get(f'https://timesofmalta.com/search?keywords=inset&author=0&tags=0&sort=date&order=desc&fields%5B0%5D=title&fields%5B1%5D=body&page={str(page_index+2)}') #Skip page

#             try: driver.find_element(By.XPATH,'//*[@id="qc-cmp2-ui"]/div[2]/div/button[2]').click() #Close pop-up
#             except: pass

#             article_index = 0 #Invoke next-page switch
            
#             continue #Go back to beggining

        

#         page_index   += 1
#         article_index = 0
#         print(f'Next Page! -> {page_index+1}')
    
#     except selenium.common.exceptions.ElementClickInterceptedException as e:
#         print('Click Intercepted - Skipping')
#         article_index += 1
#         continue

#     except Exception as e:
#         print(str(e))
#         article_index += 1
#         page_index += 1
#         continue